In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import tensorflow as tf

In [3]:
# Note: The format of data is .npy, which means numpy arrays.
DATA_PATH = os.path.join("/home/raspberrypi/Downloads/Data")

# Actions that we try to detect
actions = np.array(["HOW ARE YOU","THANKS","WORK","PLEASE","SORRY", "HELLO", "AFTERNOON", "DEAF", "GOOD", "BAD"])


no_sequences = 40


sequence_length = 20

In [4]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [6]:
label_map

{'HOW ARE YOU': 0,
 'THANKS': 1,
 'WORK': 2,
 'PLEASE': 3,
 'SORRY': 4,
 'HELLO': 5,
 'AFTERNOON': 6,
 'DEAF': 7,
 'GOOD': 8,
 'BAD': 9}

In [7]:
np.array(sequences).shape

(400, 20, 258)

In [8]:
X = np.array(sequences)

In [9]:
y = to_categorical(labels).astype(int)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 30)

In [11]:
y_test.shape

(120, 10)

# Modelling

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [13]:
# Designing Neural Network
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

/home/raspberrypi/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Compiling
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [ ]:
# Fitting
model.fit(X_train, y_train, epochs=20)

Epoch 1/20


In [ ]:
# Summary
model.summary()

# Evaluation of model on test data

In [ ]:
res = model.predict(X_test)

In [ ]:
# Saving the trained Model
model.save('isl6.h5')

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(ytrue, yhat,average='macro')

In [ ]:
recall_score(ytrue, yhat,average='macro')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss: {}, Test Accuracy: {}'.format(loss, accuracy))